# Partioning the evolution of uncertainties

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd

import os
from glob import glob
from oggm import utils
from tqdm import tqdm
from tqdm.notebook import tqdm

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl     = px.colors.qualitative.D3
os.chdir('/home/rooda/OGGM_results/')

import warnings
warnings.simplefilter("ignore")

# Ids for each glacier

In [ ]:
RGI6_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")
RGI7_ids = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")
RGI6_ids = RGI6_ids[RGI6_ids.area_km2 > 7][["RGIId", "Zone"]]
RGI7_ids = RGI7_ids[RGI7_ids.area_km2 > 7]

# RGI6 doesnt have IDs yet 
RGI7_ids = utils.cook_rgidf(RGI7_ids, o1_region='17', o2_region='02', bgndate= RGI7_ids.src_date, 
                            version = "70", assign_column_values= {'Zone' : 'Zone'})

RGI7_ids = RGI7_ids[["RGIId", "Zone"]]
ids = pd.concat([RGI6_ids, RGI7_ids]).set_index("RGIId")

dict_zone = {1:'PPY', 2:'PCA', 3:'NPI-E', 4:'NPI-W', 5:'SPI-N', 6:'SPI-C', 7:'SPI-S', 8:'GCN', 9:'CDI'}
ids = ids.replace({"Zone": dict_zone})

In [ ]:
def preprocess(ds): # remove unnecessary variables and coordinates
    return ds.drop_vars(['hydro_year', 'hydro_month', 'calendar_year', 'calendar_month'])[variables]

def hydro_variables(ds): # calculate total_runoff, melt_on_glacier and smb
    ds["total_runoff"] = ((ds.melt_off_glacier + ds.melt_on_glacier + ds.liq_prcp_off_glacier + ds.liq_prcp_on_glacier)*1e-3)/(365*86400) # m3/s
    ds["melt_on_glacier"] = ((ds.melt_on_glacier)*1e-3)/(365*86400) # m3
    ds["smb"] = (ds.volume.diff(dim = "time") * 900 / ds.area) 
    return ds[variables_final]

In [ ]:
# variables to analize
variables        = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier', 'volume', 'area']
variables_final  = ['total_runoff','melt_on_glacier', 'smb']
scenarios        = ["ssp126","ssp245","ssp370","ssp585"]

In [ ]:
dataset_mean = []
dataset_var  = []

for scenario in tqdm(scenarios): 
    all_combs = glob("/home/rooda/OGGM_results/new/*/run_output_*"+ scenario +"*.nc", recursive = True)
    all_opts   = xr.open_mfdataset(all_combs, combine='nested', concat_dim="options", chunks="auto", parallel=True, preprocess=preprocess)

    # assing zone to each glacier and aggregate the result 
    ids_subset = ids[ids.index.isin(all_opts.rgi_id.to_pandas().tolist())]
    all_opts   = all_opts.assign_coords(rgi_id = ids_subset.Zone.tolist())
    all_opts   = all_opts.groupby('rgi_id').sum()
    all_opts   = all_opts.chunk("auto")
    all_opts   = hydro_variables(all_opts)
    all_opts   = all_opts.isel(time = slice(0, -1))
    
    # standard desviation
    all_opts_var = all_opts.std(dim="options")
    all_opts_var = all_opts_var.to_dataframe()
    all_opts_var["scenario"] = scenario
    all_opts_var = all_opts_var.set_index("scenario", append=True)
    all_opts_var = all_opts_var.reorder_levels(['scenario', 'rgi_id', 'time'])
    dataset_var.append(all_opts_var)
    
    # mean
    all_opts_mean = all_opts.mean(dim="options")
    all_opts_mean = all_opts_mean.to_dataframe()
    all_opts_mean["scenario"] = scenario
    all_opts_mean = all_opts_mean.set_index("scenario", append=True)
    all_opts_mean = all_opts_mean.reorder_levels(['scenario', 'rgi_id', 'time'])
    dataset_mean.append(all_opts_mean)
    
dataset_mean = pd.concat(dataset_mean)
dataset_var  = pd.concat(dataset_var)

In [ ]:
variable = "total_runoff"
dict_names ={"melt_on_glacier": "Melt on glacier (m<sup>3</sup> s<sup>-1</sup>)", 
             "total_runoff"   : "Total runoff (m<sup>3</sup> s<sup>-1</sup>)", 
             "smb"            : "Specific mass balance (kg m<sup>-2</sup>)"}

scenarios  = ["ssp126", "ssp245", "ssp370", "ssp585"]
dict_colors = {"ssp126":cl[0], "ssp245":cl[2], "ssp370":cl[1], "ssp585":cl[3]}

basins_id  = np.array(['PPY', 'PCA','NPI-E','NPI-W','SPI-N', 'SPI-C', 'SPI-S', 'GCN', 'CDI']).reshape(3,3)
fig    = make_subplots(rows=3, cols=3, horizontal_spacing = 0.05, vertical_spacing = 0.035, shared_xaxes= True, shared_yaxes= False, 
                       subplot_titles=basins_id.reshape(1,9)[0], y_title = dict_names[variable])

time_series = dataset_mean.reset_index()
time_series_sd = dataset_var.reset_index()


for x in range(0,3):
    for y in range(0,3):
        for t in range(0,4):
        
            # legend only for first plot
            if x==0 and y==0:
                # time series
                time_series_id = time_series[time_series.rgi_id == basins_id[x,y]][time_series.scenario == scenarios[t]]
                time_series_sd_id = time_series_sd[time_series_sd.rgi_id == basins_id[x,y]][time_series_sd.scenario == scenarios[t]]
        
                fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id[variable], mode='lines', name= scenarios[t], 
                                     marker=dict(color=dict_colors[scenarios[t]]), showlegend=True, legendgroup='g1'), row=x+1, col=y+1)
            # time series
            time_series_id = time_series[time_series.rgi_id == basins_id[x,y]][time_series.scenario == scenarios[t]]
            time_series_sd_id = time_series_sd[time_series_sd.rgi_id == basins_id[x,y]][time_series_sd.scenario == scenarios[t]]

            # mean value
            fig.add_trace(go.Scatter(x=time_series_id.time, y=time_series_id[variable], mode='lines', name= scenarios[t], 
                                     marker=dict(color=dict_colors[scenarios[t]]), showlegend=False, legendgroup='g1'), row=x+1, col=y+1)
            
            # uncertainty: +-1 sd
            fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=time_series_id[variable]+time_series_sd_id[variable], line=dict(width=0),
                             fillcolor='rgba(0, 0, 0, 0.05)', showlegend=False, legendgroup='g1'), row=x+1, col=y+1)
            fig.add_trace(go.Scatter(x=time_series_sd_id.time, y=time_series_id[variable]-time_series_sd_id[variable], line=dict(width=0),
                             fillcolor='rgba(0, 0, 0, 0.05)', fill='tonexty', showlegend=False, legendgroup='g1'), row=x+1, col=y+1)

fig.update_yaxes(ticks="outside")
fig.update_layout(legend=dict(yanchor="top", y=0.45, xanchor="left", x=0.35, orientation="v", bgcolor = 'rgba(0,0,0,0.0)'))
fig.update_layout(height=800, width=1100, template = "seaborn", margin = dict(l=100, r=20, b=20, t=20))
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_ts_" + variable + ".png", scale=4)
fig.show()